# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [170]:
from IPython.core.display_functions import display

# С сайта выгрузил после 3 задачи
"""url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))"""
df=pd.read_csv('Химический анализ родника в Нескучном саду.csv', sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [171]:
# Решение

# Формирование столбца "Максимум"
df['Максимум'] = [re.search('(\d+)?,?\d+$', x).group()
                  for x in df['Норматив']]
df['Максимум'] = [re.sub(',', '.', x)
                  for x in df['Максимум']]
df['Максимум'] = pd.to_numeric(df['Максимум'],
                               downcast='float')

# Формирование столбца "Минимум"
df['Минимум'] = [re.sub('^[не].*', '0', x)
                 for x in df['Норматив']]
df['Минимум'] = [re.search('(\d+)?,?\d+', x).group()
                 for x in df['Минимум']]
df['Минимум'] = pd.to_numeric(df['Минимум'],
                              downcast='float')

# Перевод столбца "Результат анализа" в числовой формат
df['Результат анализа'] = [re.sub('б/цвета', '0', x)
                 for x in df['Результат анализа']]
df['Результат анализа'] = pd.to_numeric(df['Результат анализа'],
                                            downcast='float',
                                            errors='coerce')

df

,Показатель,Единица измерений,Результат анализа,Норматив,Максимум,Минимум
0,pH,единицы pH,8.400,в пределах 6-9,9.0,6.0
1,Запах,баллы,1.000,не более 2-3,3.0,0.0
2,Цветность,градусы,0.000,не более 30,30.0,0.0
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,10.0,7.0
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",1.5,0.0
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",3.3,0.0
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,45.0,0.0
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",3.5,0.0
8,Хлориды (Cl),мг/дм3,200.000,не более 350,350.0,0.0
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,500.0,0.0


In [172]:
# Сверка результатов с нормативами
df['Вывод'] = 'в норме'
for i in range(len(df)):
    if df.iloc[i]['Результат анализа'] > df.iloc[i]['Максимум']\
            or df.iloc[i]['Результат анализа'] < df.iloc[i]['Минимум']:
        df.iloc[i, df.columns.get_loc('Вывод')] = "не в норме"

# Удаление вспомогательных столбцов
df.drop(columns = ['Максимум', 'Минимум'], axis = 1, inplace = True)

# Установка индекса на столбец "Показатель"
df = df.set_index(['Показатель'])
df

,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,в норме
Запах,баллы,1.000,не более 2-3,в норме
Цветность,градусы,0.000,не более 30,в норме
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,в норме
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",в норме
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",в норме
Нитраты (по NO3),мг/дм3,24.000,не более 45,в норме
Фосфаты (P),мг/дм3,0.360,"не более 3,5",в норме
Хлориды (Cl),мг/дм3,200.000,не более 350,в норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [258]:
# Решение

# Формирование списка с фруктами
box = ['апельсин'] * 5 + ['яблоко'] * 4

N = 10000 # Количество экспериментов
M = 0 # Количество успешных экспериментов

# Выборка без повторений по 3 фрукта
for i in range(0, N):
    if list(np.random.choice(box, 3, replace=False)) == ['апельсин'] * 3:
        M+=1

# Вычисление вероятности (ответ ~совпадает с аналитическим)
P = M/N
P

0.1194

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [287]:
# # Решение

# Формирование списка с деталями
details = ['std'] * 7 + ['no_std'] * 3

N = 10000 # Количество экспериментов
M = 0 # Количество успешных экспериментов

# Выборка без повторений по 2 детали
for i in range(0, N):
    check_detail = list(np.random.choice(details, 2, replace=False))
    if check_detail == ['no_std','std']:
        M+=1

# Вычисление вероятности (ответ ~совпадает с аналитическим)
P = M/N
P

0.2343

### Задача 1 (загрузка с сайта). Программа отработала эквивалентно, как и напрямую с csv

In [288]:
from IPython.core.display_functions import display

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [289]:
# Решение

# Формирование столбца "Максимум"
df['Максимум'] = [re.search('(\d+)?,?\d+$', x).group()
                  for x in df['Норматив']]
df['Максимум'] = [re.sub(',', '.', x)
                  for x in df['Максимум']]
df['Максимум'] = pd.to_numeric(df['Максимум'],
                               downcast='float')

# Формирование столбца "Минимум"
df['Минимум'] = [re.sub('^[не].*', '0', x)
                 for x in df['Норматив']]
df['Минимум'] = [re.search('(\d+)?,?\d+', x).group()
                 for x in df['Минимум']]
df['Минимум'] = pd.to_numeric(df['Минимум'],
                              downcast='float')

# Перевод столбца "Результат анализа" в числовой формат
df['Результат анализа'] = [re.sub('б/цвета', '0', x)
                 for x in df['Результат анализа']]
df['Результат анализа'] = pd.to_numeric(df['Результат анализа'],
                                            downcast='float',
                                            errors='coerce')

df

,Показатель,Единица измерений,Результат анализа,Норматив,Максимум,Минимум
0,pH,единицы pH,8.400,в пределах 6-9,9.0,6.0
1,Запах,баллы,1.000,не более 2-3,3.0,0.0
2,Цветность,градусы,0.000,не более 30,30.0,0.0
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,10.0,7.0
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",1.5,0.0
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",3.3,0.0
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,45.0,0.0
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",3.5,0.0
8,Хлориды (Cl),мг/дм3,200.000,не более 350,350.0,0.0
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,500.0,0.0


In [290]:
# Сверка результатов с нормативами
df['Вывод'] = 'в норме'
for i in range(len(df)):
    if df.iloc[i]['Результат анализа'] > df.iloc[i]['Максимум']\
            or df.iloc[i]['Результат анализа'] < df.iloc[i]['Минимум']:
        df.iloc[i, df.columns.get_loc('Вывод')] = "не в норме"

# Удаление вспомогательных столбцов
df.drop(columns = ['Максимум', 'Минимум'], axis = 1, inplace = True)

# Установка индекса на столбец "Показатель"
df = df.set_index(['Показатель'])
df

,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,в норме
Запах,баллы,1.000,не более 2-3,в норме
Цветность,градусы,0.000,не более 30,в норме
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,в норме
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",в норме
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",в норме
Нитраты (по NO3),мг/дм3,24.000,не более 45,в норме
Фосфаты (P),мг/дм3,0.360,"не более 3,5",в норме
Хлориды (Cl),мг/дм3,200.000,не более 350,в норме
